<a href="https://colab.research.google.com/github/JHWannabe/Python-AI_5/blob/main/0218%20Day3/3_%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98%EB%9E%802.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Pytorch로 구현한 손글씨
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import numpy as np

In [16]:
# 공개 데이터셋에서 학습 데이터를 내려받음
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# 공개 데이터셋에서 테스트 데이터를 내려받음
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

In [17]:
batch_size = 64

# 데이터로더를 생성, tensor에서는 dataset
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print('Shaper of X [N, C, H, W] : ', X.shape)
  print('Shaper of y : ', y.shape, y.dtype)
  break

Shaper of X [N, C, H, W] :  torch.Size([64, 1, 28, 28])
Shaper of y :  torch.Size([64]) torch.int64


In [18]:
# 학습에 사용할 CPU나 GPU장치를 얻음
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Using {} device".format(device))

Using cuda device


In [24]:
# Model 정의
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 128),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(128, 10)
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
  
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)


In [25]:
# Loss 함수와 Optimizer 설정
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [26]:
# Training을 위한 함수
def train(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    # 예측 오류 계산
    pred = model(X)
    loss = loss_fn(pred, y)

    # 역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [27]:
# Test를 위한 함수
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  print(size)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [28]:
epochs = 10
for t in range(epochs):
  print(f'Epoch {t+1}\----------------------------')
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print('끝')

Epoch 1\----------------------------
loss: 2.293231 [    0/60000]
loss: 0.486417 [ 6400/60000]
loss: 0.326116 [12800/60000]
loss: 0.423079 [19200/60000]
loss: 0.236997 [25600/60000]
loss: 0.337470 [32000/60000]
loss: 0.178601 [38400/60000]
loss: 0.387965 [44800/60000]
loss: 0.254737 [51200/60000]
loss: 0.370258 [57600/60000]
10000
Test Error: 
 Accuracy: 93.5%, Avg loss: 0.207715 

Epoch 2\----------------------------
loss: 0.151991 [    0/60000]
loss: 0.173013 [ 6400/60000]
loss: 0.097062 [12800/60000]
loss: 0.192091 [19200/60000]
loss: 0.168317 [25600/60000]
loss: 0.228615 [32000/60000]
loss: 0.068405 [38400/60000]
loss: 0.213916 [44800/60000]
loss: 0.159034 [51200/60000]
loss: 0.203115 [57600/60000]
10000
Test Error: 
 Accuracy: 95.5%, Avg loss: 0.142188 

Epoch 3\----------------------------
loss: 0.099348 [    0/60000]
loss: 0.109225 [ 6400/60000]
loss: 0.070877 [12800/60000]
loss: 0.103522 [19200/60000]
loss: 0.108221 [25600/60000]
loss: 0.168911 [32000/60000]
loss: 0.052397 [384